In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

In [2]:
inputs_file = '../results/inputs.npy'
targets_file = '../results/targets.npy'
preds_file = '../results/preds.npy'

In [3]:
inputs = np.load(inputs_file)
targets = np.load(targets_file)
preds = np.load(preds_file)

In [4]:
inputs.shape, targets.shape, preds.shape

((5860071, 4), (5860071, 2), (5860071, 2))

In [5]:
input_mean = np.array([1523.10199582, 1729.62046887,   26.72577967,  316.5513978])
input_std = np.array([569.13566635, 566.33636362,  14.90088159, 183.2160048 ])

inputs = inputs * input_std + input_mean

In [6]:
output_mean = np.array([3.29223762e-01, 1.78798090e+04])
output_std = np.array([2.17284490e-01, 1.51990336e+04])

targets = targets * output_std + output_mean

In [7]:
preds = preds * output_std + output_mean

In [8]:
timesteps = np.unique(inputs[:, -1])
timesteps.shape

(573,)

In [9]:
for t in timesteps:
    
    inputs_init = inputs[inputs[:, -1]==t]
    targets_init = targets[inputs[:, -1]==t]
    preds_init = preds[inputs[:, -1]==t]
    
    fig = plt.figure(figsize=(16, 6))
    ax1 = fig.add_subplot(1, 2, 1, projection='3d')
    ax2 = fig.add_subplot(1, 2, 2, projection='3d')
    
    im1 = ax1.scatter(inputs_init[:, 0], inputs_init[:, 1], inputs_init[:, 2], c=preds_init[:, 0], vmin=targets_init[:, 0].min(), vmax=targets_init[:, 0].max())
    ax1.set_title('Hydraulic-Head Predictions')
    
    im2 = ax2.scatter(inputs_init[:, 0], inputs_init[:, 1], inputs_init[:, 2], c=targets_init[:, 0])
    ax2.set_title('Hydraulic-Head Targets')
    
    fig.colorbar(im1)
    fig.colorbar(im2)
    
    fig.savefig(f'../results/plots/head/{t:.2f}.png', bbox_inches='tight')

    plt.clf()
    plt.close()

In [10]:
import cv2
import os


In [11]:
image_folder = '../results/plots/head/'
video_name = '../results/plots/head.avi'

images = [img for img in os.listdir(image_folder) if img.endswith(".png")]
frame = cv2.imread(os.path.join(image_folder, images[0]))
height, width, layers = frame.shape

video = cv2.VideoWriter(video_name, 0, 4, (width, height))

for image in images:
    video.write(cv2.imread(os.path.join(image_folder, image)))

cv2.destroyAllWindows()
video.release()